# TF, IDF and TFIDF for Keyword Extraction
https://kavita-ganesan.com/extracting-keywords-from-text-tfidf/

In [ ]:
%%bash
test -e parlament_transcriptions_tokenized_de.tsv || wget https://files.ifi.uzh.ch/cl/siclemat/lehre/hs19/tm/parlament_transcriptions_tokenized_de.tsv -O parlament_transcriptions_tokenized_de.tsv

grep -P "SozialdemokratischeFraktion|FraktionderSchweizerischenVolkspartei" < parlament_transcriptions_tokenized_de.tsv \
 | cut -f 1,3 \
 | head -n 8000 > sp_svp.tsv

wc -l sp_svp.tsv

In [ ]:
import pandas as pd
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
df = pd.read_csv("sp_svp.tsv", sep="\t",header=None,names=['party','text'])

In [ ]:
df.head()

In [ ]:
#create a vocabulary of frequent words, 
#ignore words that appear in max_df of documents, 
cv = CountVectorizer(max_df=0.30, max_features=10000, stop_words=["es"])
word_count_vector = cv.fit_transform(df['text'])

The doc x one-hot-encoded words matrix.

In [ ]:
word_count_vector

In [ ]:
cv.stop_words_

In [ ]:
list(cv.vocabulary_.keys())[:40]

In [ ]:
tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

In [ ]:
def sort_coo(coo_matrix):
    """Sort in descending order"""
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [ ]:
feature_names = cv.get_feature_names()

In [ ]:
# get a document that we want to extract keywords from
# could be some new text... but we are lazy ...
docs = df.sample(n=1)

#generate tf-idf for the given document
tf_idf_vector = tfidf_transformer.transform(cv.transform([docs.iloc[0]["text"]]))

#sort the tf-idf vectors by descending order of scores
sorted_items = sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 5
keywords = extract_topn_from_vector(feature_names, sorted_items, 5)

# report
print(docs.iloc[0]["party"])
print(docs.iloc[0]["text"])
print("\nKEYWORDS")
for k,v in keywords.items():
    print(f"{k}: {v}")

## What could be improved? How could this be used?